<a href="https://colab.research.google.com/github/trungphan9x/ML_project1/blob/master/trung_colab_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS582 Machine Learning - Mini Project 1

Professor: Anthony Sander

Team:


*   Khoa Nam Nguyen
*   Thai Trung Phan

Dataset: https://www.kaggle.com/camnugent/california-housing-prices



# Loading all libraries

In [1]:
!pip uninstall scikit-learn
!pip install scikit-learn==0.24.0
!pip install auto-sklearn

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from pandas.plotting import scatter_matrix
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer



from autosklearn.experimental.askl2 import AutoSklearn2Classifier
import autosklearn.classification
import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics


Usage:   
  pip3 uninstall [options] <package> ...
  pip3 uninstall [options] -r <requirements file> ...

no such option: -f


In [ ]:
!pip install lime

In [ ]:
!pip install shap

# STEP 1: LOADING DATASET

Min of 500 samples , must include at least one categorical feature

In [ ]:
!git clone https://github.com/trungphan9x/ML_project1.git

In [ ]:
housing = pd.read_csv('/content/ML_project1/dataset/Cali_housing_prices/housing.csv')

# STEP 2: EDA

# Identify if there are any missing values


> There are 20,640 instances in the dataset. Notice that the total_bed rooms attribute has only 20,433 nonnull values, meaning that 207 districts are missing this feature. We will need to take care of this.






In [ ]:
housing.info()

# Identify categorical feature



> All attributes are numerical, except the ocean_proximity field. When you looked at the top five rows, you probably noticed that the values in the ocean_proximity column were repetitive, which means that it is probably a categorical attribute 

In [ ]:
housing.head()



> Check what categories exist and how many districts belong to each category



In [ ]:
housing['ocean_proximity'].value_counts()

# Summary of the numerical atributes

In [ ]:
housing.describe()

# Ploting a histogram of each numerical atribute

In [ ]:
housing.hist(bins=50, figsize=(20,15)) 
plt.show()



> There are a few things you might notice in these histograms:


1.   The median house value is our target attribute (our labels).
2.   The median income attribute: the numbers represent roughly tens of thousands of dollars (e.g., 3 actually means about $30,000).
3.   These attributes have very different scales.
4.   Finally, many histograms are tail-heavy: they extend much farther to the right of the median than to the left. This may make it a bit harder for some Machine Learning algorithms to detect patterns. We will try transforming these attributes later on to have more bell-shaped distributions.






# Visualizing Geographical Data

Create a scatterplot of all districts to visualize the data

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude")

This looks like California actually. Setting the alpha option to 0.1 makes it much easier to visualize the places where there is a high density of data points:

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.1)

Now that’s much better: you can clearly see the high-density areas, namely the Bay Area and around Los Angeles and San Diego, plus a long line of fairly high density in the Central Valley, in particular around Sacramento and Fresno.


Now let’s look at the housing prices (Figure 2-13). The radius of each circle represents the district’s population (option s), and the color represents the price (option c). We will use a predefined color map (option cmap) called jet, which ranges from blue (low values) to red (high prices):

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.4,
             s=housing["population"]/100, label="population", figsize=(10,7),
             c="median_house_value", cmap=plt.get_cmap("jet"), colorbar=True,
             sharex=False)
plt.legend()

This image tells you that the housing prices are very much related to the location (e.g., close to the ocean) and to the population density. A clustering algorithm should be useful for detecting the main cluster and for adding new features that measure the proximity to the cluster centers. The ocean proximity attribute may be useful as well, although in Northern California the housing prices in coastal districts are not too high, so it is not a simple rule.

# Correlations

In [ ]:
corr_matrix = housing.corr()

Now let’s look at how much each attribute correlates with the median house value:

In [ ]:
corr_matrix["median_house_value"].sort_values(ascending=False)

When it is close to 1, it means that there is a strong positive correlation; for example, the median house value tends to go up when the median income goes up

When the coefficient is close to –1, it means that there is a strong negative correlation; you can see a small negative correlation between the latitude and the median house value (i.e., prices have a slight tendency to go down when you go north).

Another way to check for correlation between attributes is to use the pandas scatter_matrix() function

In [ ]:
attributes = ["median_house_value", "median_income", "total_rooms","housing_median_age"]
scatter_matrix(housing[attributes], figsize=(12, 8))

In [ ]:
 housing.plot(kind="scatter", x="median_income", y="median_house_value", alpha=0.1)

# Atribute Combination

In [ ]:
housing["rooms_per_household"] = housing["total_rooms"]/housing["households"]
housing["bedrooms_per_room"] = housing["total_bedrooms"]/housing["total_rooms"]
housing["population_per_household"]=housing["population"]/housing["households"]

In [ ]:
corr_matrix = housing.corr()

In [ ]:
corr_matrix["median_house_value"].sort_values(ascending=False)

The new bedrooms_per_room attribute is much more correlated with the median house value than the total number of rooms or bedrooms.

Apparently houses with a lower bedroom/room ratio tend to be more expensive. 

The larger the houses, the more expensive they are.

# STEP 3: DATA PREPARATION

# Handle the categorical columns

In [ ]:
#le = LabelEncoder()
#housing['ocean_proximity'] = le.fit_transform(housing['ocean_proximity'])
housing = pd.get_dummies(housing, columns=['ocean_proximity'])


In [ ]:
housing

#  Processing missing values

Create a SimpleImputer instance, specifying that you want to replace each attribute’s missing values with the median of that attribute:

In [ ]:
imputer = SimpleImputer(strategy="median")

Fit the imputer instance to the training data using the fit() method:

In [ ]:
imputer.fit(housing)

In [ ]:
imputer.statistics_

Use this “trained” imputer to transform the training set by replacing missing values with the learned medians:

In [ ]:
X = imputer.transform(housing)

The result is a plain NumPy array containing the transformed features. If you want to put it back into a pandas DataFrame, it’s simple:

In [ ]:
housing = pd.DataFrame(X, columns=housing.columns, index=housing.index)
housing

In [ ]:
housing.info()

# Convert Regression dataset to Binary Classification dataset

In [ ]:
housing['median_house_value'] = np.where(housing['median_house_value']>200000.0,1,0)
housing

In [ ]:
housing['median_house_value'].value_counts()

# Create trainset and testset

The median income is a very important attribute to predict median housing prices. We may want to ensure that the test set is representative of the various categories of incomes in the whole dataset. So we are gonna create an income category attribute with five categories:

In [ ]:
housing["income_cat"] = pd.cut(housing["median_income"],bins=[0., 1.5, 3.0, 4.5, 6., np.inf],labels=[1, 2, 3, 4, 5])

In [ ]:
 housing["income_cat"].hist()

In [ ]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

In [ ]:
for train_index, test_index in split.split(housing, housing["income_cat"]):
        strat_train_set = housing.loc[train_index]
        strat_test_set = housing.loc[test_index]

In [ ]:
strat_test_set["income_cat"].value_counts() / len(strat_test_set)

In [ ]:
strat_train_set["income_cat"].value_counts() / len(strat_train_set)

Then remove the income_cat attribute so the data is back to its original

In [ ]:
for set_ in (strat_train_set, strat_test_set): 
  set_.drop("income_cat", axis=1, inplace=True)

Separate the predictors and the labels, since we don’t necessarily want to apply the same transformations to the predictors and the target values:

In [ ]:
housing = strat_train_set.drop("median_house_value", axis=1)
housing_labels = strat_train_set["median_house_value"].copy()

In [ ]:
X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()

In [ ]:
housing

In [ ]:
X_test

# Nomalization Data

In [ ]:
# normalize data
std_scaler = StandardScaler()

housing_prepared = std_scaler.fit_transform(housing)
X_test_prepared = std_scaler.fit_transform(X_test)

In [ ]:
housing_prepared = pd.DataFrame(housing_prepared, columns=housing.columns, index=housing.index)
housing_prepared

In [ ]:
X_test_prepared = pd.DataFrame(X_test_prepared, columns=X_test.columns, index=X_test.index)
X_test_prepared

# STEP 4: MODEL TUNING

Using GridSearchCV to find a great combination of hyperparameter values

In [ ]:
from sklearn.model_selection import GridSearchCV
def best_estimator(estimator,param_grid,X,y,cv):
  grid = GridSearchCV(estimator, param_grid, cv = cv, scoring = 'accuracy', n_jobs=-1)
  grid.fit(X,y)
  best_estimator = grid.best_estimator_
  best_score = grid.best_score_
  print("Best Params:", grid.best_params_)
  print("Best Score:", grid.best_score_)
  print("Best Estimator:", grid.best_estimator_)
  return best_estimator, best_score

Plot learning curve: `https://scikit-learn.org/stable/auto_examples/model_selection/plot_learning_curve.html`

In [ ]:
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve


def plot_learning_curve(estimator, title, X, y, axe=None, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    
    if axe is None:
        _, axe = plt.subplots(figsize=(20, 5))

    plt.title("Learning Curve with "+ title)
    axe.set_xlabel("Training examples")
    axe.set_ylabel("Score")

    train_sizes, train_scores, test_scores, fit_times, _ = \
        learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs,
                       train_sizes=train_sizes,
                       return_times=True)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)

    # Plot learning curve
    axe.grid()
    axe.fill_between(train_sizes, train_scores_mean - train_scores_std,
                         train_scores_mean + train_scores_std, alpha=0.1,
                         color="r")
    axe.fill_between(train_sizes, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1,
                         color="g")
    axe.plot(train_sizes, train_scores_mean, 'o-', color="r",
                 label="Training score")
    axe.plot(train_sizes, test_scores_mean, 'o-', color="g",
                 label="Cross-validation score")
    axe.legend(loc="best")

    return plt


Plot validation curve: `https://scikit-learn.org/stable/auto_examples/model_selection/plot_validation_curve.html`

In [ ]:

from sklearn.model_selection import validation_curve

def plot_validation_curve(estimator, title, X, y, param_name=None, param_range=None, cv= None, n_jobs=None):
  train_scores, test_scores = validation_curve(
      estimator, X, y, param_name=param_name, param_range=param_range,
      scoring="accuracy", n_jobs=1)
  train_scores_mean = np.mean(train_scores, axis=1)
  train_scores_std = np.std(train_scores, axis=1)
  test_scores_mean = np.mean(test_scores, axis=1)
  test_scores_std = np.std(test_scores, axis=1)
  plt.subplots(figsize=(20, 5))
  plt.title("Validation Curve with "+ title)
  plt.xlabel(param_name)
  plt.ylabel("Score")
  plt.ylim(0.0, 1.1)
  lw = 2
  plt.semilogx(param_range, train_scores_mean, label="Training score",
              color="darkorange", lw=lw)
  plt.fill_between(param_range, train_scores_mean - train_scores_std,
                  train_scores_mean + train_scores_std, alpha=0.2,
                  color="darkorange", lw=lw)
  plt.semilogx(param_range, test_scores_mean, label="Cross-validation score",
              color="navy", lw=lw)
  plt.fill_between(param_range, test_scores_mean - test_scores_std,
                  test_scores_mean + test_scores_std, alpha=0.2,
                  color="navy", lw=lw)
  plt.legend(loc="best")
  plt.show()


# 1) SVC

In [ ]:
from sklearn.svm import SVC
param_svc = {
    'C': [0.001, 0.1, 1, 10, 100] #regularization factor
}
svc_estimator, svc_score = best_estimator(SVC(), param_svc, housing_prepared, housing_labels, 5) 

In [ ]:
plot_learning_curve(svc_estimator, "SVC", housing_prepared, housing_labels)

In [ ]:
plot_validation_curve(svc_estimator, "SVC", housing_prepared, housing_labels, param_name="C", param_range=[0.001, 0.1, 1, 10, 100])

# 2) KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
k_range = np.arange(1,30)
param_knn = dict(n_neighbors = k_range)
knn_estimator, knn_score = best_estimator(KNeighborsClassifier(), param_knn, housing_prepared, housing_labels, 5)

In [ ]:
plot_learning_curve(knn_estimator, "KNN", housing_prepared, housing_labels)

In [ ]:
param_range = [1,3,7,14,20]
plot_validation_curve(knn_estimator, "KNN", housing_prepared, housing_labels, param_name="n_neighbors", param_range = param_range)

# 3) Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
log_params = {
    'C': np.logspace(-3,3,7),
    'penalty':['l1','l2']# l1 lasso l2 ridge
}
log_estimator,log_score = best_estimator(LogisticRegression(max_iter=1000), log_params,housing_prepared, housing_labels,5)

In [ ]:
plot_learning_curve(log_estimator, "Logistic Regression", housing_prepared, housing_labels)

In [ ]:
param_range = np.logspace(-3,3,7)
plot_validation_curve(log_estimator, "Logistic Regression", housing_prepared, housing_labels, param_name="C", param_range = param_range)

# 4) Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtree_params = {
     'max_depth':np.arange(1,20)
}
dt_estimator,dt_score = best_estimator(DecisionTreeClassifier(), dtree_params, housing_prepared, housing_labels, 5)

In [ ]:
plot_learning_curve(dt_estimator, "Decision Tree", housing_prepared, housing_labels)

In [ ]:
param_range = [1,3,8,12,20]
plot_validation_curve(dt_estimator, "Decision Tree", housing_prepared, housing_labels, param_name="max_depth", param_range = param_range)

# 5) MLP

`https://scikit-learn.org/stable/modules/neural_networks_supervised.html`

In [ ]:
from sklearn.neural_network import MLPClassifier
mlp_params = {
    'hidden_layer_sizes': [(5,),(10,),(20,)]
}
mlp_estimator, mlp_score = best_estimator(MLPClassifier(max_iter=1000), mlp_params, housing_prepared, housing_labels, 5)

In [ ]:
plot_learning_curve(mlp_estimator, "MLP", housing_prepared, housing_labels)

In [ ]:
param_range = [1, 20, 50,100]
plot_validation_curve(mlp_estimator, "MLP", housing_prepared, housing_labels, param_name="hidden_layer_sizes", param_range = param_range)

# 6) Ensemble - Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rForest = [
        {'n_estimators': [3, 10, 30, 50, 80, 100, 500], 'max_features': [2, 4, 6, 8, 10, 12]},
        {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
]
rf_estimator, rf_score = best_estimator(RandomForestClassifier(), rForest, housing_prepared, housing_labels, 5)

In [ ]:
plot_learning_curve(rf_estimator, "Random Forest Classifier", housing_prepared, housing_labels)

In [ ]:
param_range = [3, 10, 30, 50, 80, 100, 500]
plot_validation_curve(rf_estimator, "Random Forest Classifier", housing_prepared, housing_labels, param_name="n_estimators", param_range = param_range)

# 7) Voting Classifier

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

eclf = VotingClassifier(voting='hard', 
                        estimators=[('svc', svc_estimator), ('knn', knn_estimator), ('log', log_estimator), ('dt', dt_estimator), ('mlp', mlp_estimator), ('rf', rf_estimator)] )
eclf = eclf.fit(housing_prepared, housing_labels)
y_pred_voting = eclf.predict(X_test_prepared)
voting_score = accuracy_score(y_test, y_pred_voting)
print(f"Voting Accuracy Score = {voting_score}")

# 8) AutoML

In [ ]:
#Auto-sklearn 2.0 includes latest research on automatically configuring the AutoML system itself and contains a multitude of improvements which speed up the fitting the AutoML system.
# from autosklearn.experimental.askl2 import AutoSklearn2Classifier
# import autosklearn.classification
# import sklearn.model_selection
# import sklearn.datasets
# import sklearn.metrics

autoMl = autosklearn.classification.AutoSklearnClassifier()
autoMl.fit(housing_prepared, housing_labels)

y_pred_automl = autoMl.predict(X_test_prepared)
autoMl_score =  sklearn.metrics.accuracy_score(y_test, y_pred_automl)
print(f"AutoML Accuracy Score = {autoMl_score}")


# AUC Curve

`https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html`

In [ ]:
from sklearn.metrics import roc_curve
from sklearn import metrics
def plot_roc_curve(estimators, titles, X, y, ax=None):
    if ax is None: _, ax = plt.subplots(figsize=(5, 5))

    for i, estimator in enumerate(estimators):
      y_pred = estimator.predict(X)
      fpr, tpr, _ = roc_curve(y, y_pred)
      ax.plot(fpr, tpr, label=f"{titles[i]}, AUC=" + "{:.2f}".format(metrics.auc(fpr, tpr)))

    ax.set_title('AUC curve')
    ax.legend(loc='best')
    ax.set_xlabel('False positive rate')
    ax.set_ylabel('True positive rate') 
    ax.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    return plt

estimators = [svc_estimator, knn_estimator, log_estimator, dt_estimator, mlp_estimator, rf_estimator]
titles = ['SVC','KNN','LOG','DT','MLP', 'RF']
plot_roc_curve(estimators, titles, X_test_prepared, y_test)
plt.show()


# SHAP

`https://www.explorium.ai/blog/interpretability-and-explainability-part-2/`

In [ ]:
import shap

row = 4
data_for_prediction = X_test_prepared.iloc[row]  # use 1 arbitrary row of data
data_for_prediction_array = data_for_prediction.values.reshape(1, -1)

explainer = shap.TreeExplainer(dt_estimator)
shap_values = explainer.shap_values(data_for_prediction)
# The shap_values is a list with two arrays. It’s cumbersome to review raw arrays, but the shap package has a nice way to visualize the results.

shap.initjs()
shap.force_plot(explainer.expected_value[1], shap_values[1], data_for_prediction)

In [ ]:
y_test